In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, 'venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# so that we can import `tools.py`
# Navigate two directories up from the current working directory
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# Add this directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pickle
from tqdm import tqdm

In [3]:
# loading a sample from our TinyStories dataset
from tools import get_data_loader
data_loader = get_data_loader(batch_size=10_000, split='train')

batch = next(iter(data_loader))
print(len(batch), batch[0])

Found cached dataset json (/Users/tunadorable/.cache/huggingface/datasets/noanabeshima___json/noanabeshima--TinyStoriesV2-226173b7dd235c68/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


10000 One day, a little girl saw a great window. She decided to join it and started to jump up and down excitedly. Suddenly, she tripped and fell and the window shattered. Everyone was amazed at the crash, but the girl was sad.
The next day, the girl heard someone knocking at her door. She opened it and found a man with a box. He asked her if she was the one who joined the window. The girl nodded sadly. The man said that he was the one who made the window and he had a surprise for her.
He opened the box and there was a new window inside! He explained that he heard that the old one had broken and wanted to make her a new one. Everyone applauded and the girl was so happy.
The girl was happy that the man made a new window for her. She thanked him and her day was great.


In [4]:
# turn it into one string instead of a list of strings
combined_string = '\n\n'.join(batch)

# find the unique characters
chars = sorted(list(set(combined_string)))
# this is the largest set of characters i found earlier from a batch size of 1_000_000, total 95 characters
if len(chars) < 95:
    chars = ['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
v = len(chars)
print('\n', chars, v)


 ['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~'] 95


# BPE tokenization
rather than actual byte-pair encoding, we'll do character-pair encoding, meaning we'll build off our base number of characters isntead of actual bytes

In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
char_encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers

In [6]:
vocab_size = 8192 # the desired final vocabulary size
num_merges = vocab_size - v

In [7]:
# most models work off bytes, but we'll be simplifying to just the index of each unique character
base_indices = char_encode(chars)
print(base_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94]


In [8]:
def classify_char(c):
    if c.isupper():
        return ["upper_letter"]
    elif c.islower():
        return ["lower_letter"]
    elif c.isdigit():
        return ["digit"]
    elif c == ' ':
        return ["space"]
    elif c == "'":
        return ["apostrophe"]
    else:
        return ["symbol"]

origin = [classify_char(ch) for ch in chars]
print(origin)

[['symbol'], ['symbol'], ['space'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['apostrophe'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['digit'], ['digit'], ['digit'], ['digit'], ['digit'], ['digit'], ['digit'], ['digit'], ['digit'], ['digit'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['upper_letter'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['symbol'], ['lower_letter'], ['lower_letter'], ['lower_letter'], ['lower_letter']

In [9]:
def can_merge(a, b):
    # Define merge rules
    rules = {
        "upper_letter": {"upper_letter", "lower_letter", "apostrophe"},
        "lower_letter": {"upper_letter", "lower_letter", "apostrophe"},
        "digit": {"digit", "space"}, 
        "symbol": {"symbol", "space"}, 
        "space": {"symbol", "digit", "apostrophe"},
        "apostrophe": {"upper_letter", "lower_letter", "space"}
    }

    # Check if all elements in a can merge with all elements in b
    for x in a:
        for y in b:
            if y not in rules[x]:
                return False
    return True

# Define test cases with expected outcomes
tests = [
    (["lower_letter", "upper_letter"], ["space"], False),
    (["lower_letter", "upper_letter"], ["digit"], False),
    (["space", "symbol"], ["space"], False),
    (["space"], ["space"], False),
    (["space", "apostrophe", "lower_letter"], ["upper_letter"], False),
    (["space", "digit"], ["symbol"], False),
    (["symbol", "space"], ["upper_letter"], False),
    (["symbol"], ["upper_letter", "space"], False),
    # Add more test cases as needed
]
    
# Run tests
for a, b, expected in tests:
    result = can_merge(a, b)
    assert result == expected, f"Test with a={a} and b={b} failed. Expected {expected}, got {result}"
    print(f"Test with a={a} and b={b} passed as expected.")

Test with a=['lower_letter', 'upper_letter'] and b=['space'] passed as expected.
Test with a=['lower_letter', 'upper_letter'] and b=['digit'] passed as expected.
Test with a=['space', 'symbol'] and b=['space'] passed as expected.
Test with a=['space'] and b=['space'] passed as expected.
Test with a=['space', 'apostrophe', 'lower_letter'] and b=['upper_letter'] passed as expected.
Test with a=['space', 'digit'] and b=['symbol'] passed as expected.
Test with a=['symbol', 'space'] and b=['upper_letter'] passed as expected.
Test with a=['symbol'] and b=['upper_letter', 'space'] passed as expected.


In [10]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    newids = []
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [11]:
tokens = char_encode(combined_string)
ids = list(tokens) # copy so we don't destroy the original list

# now let's actually do it
merges = {} # (int, int) -> int
for i in tqdm(range(num_merges)):
    stats = get_stats(ids)
    valid_pairs = {}

    # Check each pair's validity based on updated origin indices
    for pair, freq in stats.items():
        # Ensure that pair indices are within the current range of origin
        if pair[0] < len(origin) and pair[1] < len(origin):
            if can_merge(origin[pair[0]], origin[pair[1]]):
                valid_pairs[pair] = freq

    if not valid_pairs:
        break  # No more valid pairs to merge

    pair = max(valid_pairs, key=valid_pairs.get)
    idx = v + i  # Ensure this index starts from an appropriate offset
    ids = merge(ids, pair, idx)
    merges[pair] = idx
    #print(idx, origin[pair[0]], origin[pair[1]], list(set(origin[pair[0]] + origin[pair[1]])))
    origin.append(list(set(origin[pair[0]] + origin[pair[1]])))  # Ensure this is consistently updated

100%|██████████████████| 8097/8097 [1:33:23<00:00,  1.44it/s]


In [12]:
print("tokens length:", len(tokens)) # remember tokens are our original tokens
print("ids length:", len(ids)) # and ids are new tokens we've made
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 8082207
ids length: 3225806
compression ratio: 2.51X


In [13]:
# Ensure the tokenizers directory exists
if not os.path.exists('./models'):
    os.makedirs('./models')

# Prepare the tokenizer data to be saved
tokenizer_data = {
    'stoi': stoi,  # Character to integer mapping
    'merges': merges  # Merges dictionary
}

# Save the tokenizer data using pickle
with open(f'./models/{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

In [26]:
# taking a pre-existing tokenizer and trimming it down to a smaller size
# i basically ran this cell and the one below it multiple times until i got to the smallest possible size (128)
vocab_size = 95#vocab_size // 2 # 95
merges = {k: v for k, v in merges.items() if v < vocab_size}

In [27]:
# Ensure the tokenizers directory exists
if not os.path.exists('./models'):
    os.makedirs('./models')

# Prepare the tokenizer data to be saved
tokenizer_data = {
    'stoi': stoi,  # Character to integer mapping
    'merges': merges  # Merges dictionary
}

# Save the tokenizer data using pickle
with open(f'./models/{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

In [28]:
from tokenizer import load_tokenizer_data, BPE_Tokenizer
vocab_size = 8192
tokenizer_data = load_tokenizer_data(f'models/{vocab_size}.model')
tokenizer = BPE_Tokenizer(tokenizer_data['stoi'], tokenizer_data['merges'])

In [29]:
prompt = 'Once upon a time there was a boy named Tim.'
print(tokenizer.display(prompt))

Once| |upon| |a| |ti|me| |th|er|e| |was| |a| |boy| |na|me|d| |Tim|.


In [30]:
print(tokenizer.display(batch[0]))

On|e| |da|y|, |a| |li|t|tle| |gi|r|l| |saw| |a| |gr|eat| |wi|nd|ow|. |Sh|e| |decided| |to| |join| |it| |and| |start|ed| |to| |ju|m|p| |up| |and| |do|w|n| |excite|d|ly|. |Su|dd|en|ly|, |sh|e| |tr|ip|pe|d| |and| |fell| |and| |th|e| |wi|nd|ow| |sh|att|ered|. |E|ve|ry|one| |was| |amaze|d| |at| |th|e| |crash|, |bu|t| |th|e| |gi|r|l| |was| |sad|.
|Th|e| |next| |da|y|, |th|e| |gi|r|l| |hea|rd| |so|me|one| |kn|oc|ki|ng| |at| |her| |door|. |Sh|e| |open|ed| |it| |and| |fo|un|d| |a| |ma|n| |wi|th| |a| |box|. |He| |ask|ed| |her| |if| |sh|e| |was| |th|e| |one| |wh|o| |joined| |th|e| |wi|nd|ow|. |Th|e| |gi|r|l| |nod|de|d| |sadly|. |Th|e| |ma|n| |said| |th|at| |he| |was| |th|e| |one| |wh|o| |ma|de| |th|e| |wi|nd|ow| |and| |he| |had| |a| |su|r|pr|ise| |fo|r| |her|.
|He| |open|ed| |th|e| |box| |and| |th|er|e| |was| |a| |new| |wi|nd|ow| |ins|ide|! |He| |exp|la|ine|d| |th|at| |he| |hea|rd| |th|at| |th|e| |ol|d| |one| |had| |br|ok|en| |and| |wa|nt|ed| |to| |ma|ke| |her| |a| |new| |one|. |E|ve|ry|one| |app

In [31]:
for i in range(vocab_size):
    print(f"{i}: '{tokenizer.decode([i])}'")

0: '	'
1: '
'
2: ' '
3: '!'
4: '"'
5: '#'
6: '$'
7: '%'
8: '&'
9: '''
10: '('
11: ')'
12: '*'
13: '+'
14: ','
15: '-'
16: '.'
17: '/'
18: '0'
19: '1'
20: '2'
21: '3'
22: '4'
23: '5'
24: '6'
25: '7'
26: '8'
27: '9'
28: ':'
29: ';'
30: '<'
31: '='
32: '>'
33: '?'
34: 'A'
35: 'B'
36: 'C'
37: 'D'
38: 'E'
39: 'F'
40: 'G'
41: 'H'
42: 'I'
43: 'J'
44: 'K'
45: 'L'
46: 'M'
47: 'N'
48: 'O'
49: 'P'
50: 'Q'
51: 'R'
52: 'S'
53: 'T'
54: 'U'
55: 'V'
56: 'W'
57: 'X'
58: 'Y'
59: 'Z'
60: '['
61: '\'
62: ']'
63: '_'
64: '`'
65: 'a'
66: 'b'
67: 'c'
68: 'd'
69: 'e'
70: 'f'
71: 'g'
72: 'h'
73: 'i'
74: 'j'
75: 'k'
76: 'l'
77: 'm'
78: 'n'
79: 'o'
80: 'p'
81: 'q'
82: 'r'
83: 's'
84: 't'
85: 'u'
86: 'v'
87: 'w'
88: 'x'
89: 'y'
90: 'z'
91: '{'
92: '|'
93: '}'
94: '~'
95: 'he'
96: '. '
97: 'an'
98: 'the'
99: 'ed'
100: ', '
101: 'to'
102: 'and'
103: 'in'
104: 're'
105: 'it'
106: 'ou'
107: 'wa'
108: 'ha'
109: 'The'
110: 'er'
111: 'was'
112: 'en'
113: 'ay'
114: 'om'
115: 'is'
116: '.
'
117: 'ar'
118: 'on'
119: 'im'
1